In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats
import os
import plotly

In [2]:
df_bitcoin = pd.read_csv('../Data/BitcoinHeistDataWithMonthAndDate.csv')

In [3]:
df_bitcoin

,address,year,day,length,weight,count,looped,neighbors,income,label,month,date
0,111K8kZAEnJg245r2cM6y9zgJGHZtJPy6,2017,11,18,0.008333,1,0,2,1.000500e+08,princetonCerber,1,11
1,1123pJv8jzeFQaCV4w644pzQJzVWay2zcA,2016,132,44,0.000244,1,0,1,1.000000e+08,princetonLocky,5,11
2,112536im7hy6wtKbpH1qYDWtTyMRAcA2p7,2016,246,0,1.000000,1,0,2,2.000000e+08,princetonCerber,9,2
3,1126eDRw2wqSkWosjTCre8cjjQW8sSeWH7,2016,322,72,0.003906,1,0,2,7.120000e+07,princetonCerber,11,17
4,1129TSjKtx65E35GiUo4AYVeyo48twbrGX,2016,238,144,0.072848,456,0,1,2.000000e+08,princetonLocky,8,25
...,...,...,...,...,...,...,...,...,...,...,...,...
2916692,12D3trgho1vJ4mGtWBRPyHdMJK96TRYSry,2018,330,0,0.111111,1,0,1,1.255809e+09,white,11,26
2916693,1P7PputTcVkhXBmXBvSD9MJ3UYPsiou1u2,2018,330,0,1.000000,1,0,1,4.409699e+07,white,11,26
2916694,1KYiKJEfdJtap9QX2v9BXJMpz2SfU4pgZw,2018,330,2,12.000000,6,6,35,2.398267e+09,white,11,26
2916695,15iPUJsRNZQZHmZZVwmQ63srsmughCXV4a,2018,330,0,0.500000,1,0,1,1.780427e+08,white,11,26


In [4]:
df_bitcoin[df_bitcoin['year'] == 2018]['month'].value_counts()

month
5     33002
9     31001
3     31000
7     31000
8     31000
10    31000
4     30000
6     30000
1     28000
2     28000
11    26000
Name: count, dtype: int64

In [5]:
# sort by year and then by month and then by date
df_bitcoin = df_bitcoin.sort_values(by=['year', 'month', 'date'])

In [6]:
ls_year = df_bitcoin['year'].to_list()
ls_month = df_bitcoin['month'].to_list()
ls_year_month = []
for i in range(len(ls_year)):
    ym = str(ls_year[i]) + '-'
    if ls_month[i] < 10:
        ym += '0'
    ym += str(ls_month[i])
    ls_year_month.append(ym)
df_bitcoin['year_month'] = ls_year_month

In [7]:
# randomly sample 10000 rows
df_bitcoin_sample = df_bitcoin.sample(n=10000, random_state=1)

In [8]:
min_year_month = df_bitcoin['year_month'].min()
max_year_month = df_bitcoin['year_month'].max()

min_year_month_sample = df_bitcoin_sample['year_month'].min()
max_year_month_sample = df_bitcoin_sample['year_month'].max()

In [9]:
min_year_month, max_year_month, min_year_month_sample, max_year_month_sample

('2011-01', '2018-11', '2011-01', '2018-11')

In [10]:
# create new dataframe with year_month and label
df_bitcoin_year_month = df_bitcoin[['year_month', 'label']]
df_bitcoin_year_month

# for each year_month, count the number of white and non-white transactions and store in a new dataframe
df_transaction_counts = pd.DataFrame(columns=['year_month', 'white_count', 'non_white_count'])
for year_month in df_bitcoin_year_month['year_month'].unique():
    df_year_month = df_bitcoin_year_month[df_bitcoin_year_month['year_month'] == year_month]
    white_count = df_year_month[df_year_month['label'] == 'white']['label'].count()
    non_white_count = df_year_month[df_year_month['label'] != 'white']['label'].count()
    df_transaction_counts = pd.concat([df_transaction_counts, pd.DataFrame([[year_month, white_count, non_white_count]], columns=['year_month', 'white_count', 'non_white_count'])])
df_transaction_counts['proportion_ransomware'] = df_transaction_counts['non_white_count'] / (df_transaction_counts['white_count'] + df_transaction_counts['non_white_count'])

In [11]:
# create new dataframe with year_month and label
df_bitcoin_sample_year_month = df_bitcoin_sample[['year_month', 'label']]
df_bitcoin_sample_year_month

# for each year_month, count the number of white and non-white transactions and store in a new dataframe
df_transaction_counts_sample = pd.DataFrame(columns=['year_month', 'white_count', 'non_white_count'])
for year_month in df_bitcoin_sample_year_month['year_month'].unique():
    df_year_month = df_bitcoin_sample_year_month[df_bitcoin_sample_year_month['year_month'] == year_month]
    white_count = df_year_month[df_year_month['label'] == 'white']['label'].count()
    non_white_count = df_year_month[df_year_month['label'] != 'white']['label'].count()
    df_transaction_counts_sample = pd.concat([df_transaction_counts_sample, pd.DataFrame([[year_month, white_count, non_white_count]], columns=['year_month', 'white_count', 'non_white_count'])])
df_transaction_counts_sample['proportion_ransomware'] = df_transaction_counts_sample['non_white_count'] / (df_transaction_counts_sample['white_count'] + df_transaction_counts_sample['non_white_count'])

In [12]:
ls_proportion_ransomware = df_transaction_counts['proportion_ransomware'].tolist()
mean_proportion_ransomware = np.mean(ls_proportion_ransomware)
std_dev_proportion_ransomware = np.std(ls_proportion_ransomware)

ls_proportion_ransomware_sample = df_transaction_counts_sample['proportion_ransomware'].tolist()
mean_proportion_ransomware_sample = np.mean(ls_proportion_ransomware_sample)
std_dev_proportion_ransomware_sample = np.std(ls_proportion_ransomware_sample)

In [13]:
mean_proportion_ransomware, std_dev_proportion_ransomware, mean_proportion_ransomware_sample, std_dev_proportion_ransomware_sample

(0.034456267045332986,
 0.1458550560780503,
 0.012756622883689723,
 0.022453133989450393)

In [14]:
df_gtrend_bitcoin = pd.read_csv('../Data/GoogleTrends/Bitcoin.csv')
df_gtrend_crypto = pd.read_csv('../Data/GoogleTrends/Cryptocurrency.csv')
df_gtrend_ransomware = pd.read_csv('../Data/GoogleTrends/Ransomware.csv')

In [15]:
df_gtrend_bitcoin['Category: All categories']['Month']

'bitcoin: (Worldwide)'

In [16]:
gtrend_bitcoin_dict = df_gtrend_bitcoin['Category: All categories'].to_dict()
gtrend_crypto_dict = df_gtrend_crypto['Category: All categories'].to_dict()
gtrend_ransomware_dict = df_gtrend_ransomware['Category: All categories'].to_dict()

# remove first entry
gtrend_bitcoin_dict.pop('Month')
gtrend_crypto_dict.pop('Month')
gtrend_ransomware_dict.pop('Month')


'ransomware: (Worldwide)'

In [17]:
# change all values which say '<1' to 0
for key, value in gtrend_bitcoin_dict.items():
    if value == '<1':
        gtrend_bitcoin_dict[key] = 0

for key, value in gtrend_crypto_dict.items():
    if value == '<1':
        gtrend_crypto_dict[key] = 0

for key, value in gtrend_ransomware_dict.items():
    if value == '<1':
        gtrend_ransomware_dict[key] = 0

In [18]:
# change all values to int
for key, value in gtrend_bitcoin_dict.items():
    gtrend_bitcoin_dict[key] = int(value)

for key, value in gtrend_crypto_dict.items():
    gtrend_crypto_dict[key] = int(value)

for key, value in gtrend_ransomware_dict.items():
    gtrend_ransomware_dict[key] = int(value)

In [19]:
# remove keys which are not in the range of the bitcoin dataset
for key in list(gtrend_bitcoin_dict.keys()):
    min_year = int(min_year_month.split('-')[0])
    max_year = int(max_year_month.split('-')[0])
    min_month = int(min_year_month.split('-')[1])
    max_month = int(max_year_month.split('-')[1])
    year = int(key.split('-')[0])
    month = int(key.split('-')[1])
    if year < min_year or year > max_year:
        gtrend_bitcoin_dict.pop(key)
    elif year == min_year and month < min_month:
        gtrend_bitcoin_dict.pop(key)
    elif year == max_year and month > max_month:
        gtrend_bitcoin_dict.pop(key)

for key in list(gtrend_crypto_dict.keys()):
    min_year = int(min_year_month.split('-')[0])
    max_year = int(max_year_month.split('-')[0])
    min_month = int(min_year_month.split('-')[1])
    max_month = int(max_year_month.split('-')[1])
    year = int(key.split('-')[0])
    month = int(key.split('-')[1])
    if year < min_year or year > max_year:
        gtrend_crypto_dict.pop(key)
    elif year == min_year and month < min_month:
        gtrend_crypto_dict.pop(key)
    elif year == max_year and month > max_month:
        gtrend_crypto_dict.pop(key)

for key in list(gtrend_ransomware_dict.keys()):
    min_year = int(min_year_month.split('-')[0])
    max_year = int(max_year_month.split('-')[0])
    min_month = int(min_year_month.split('-')[1])
    max_month = int(max_year_month.split('-')[1])
    year = int(key.split('-')[0])
    month = int(key.split('-')[1])
    if year < min_year or year > max_year:
        gtrend_ransomware_dict.pop(key)
    elif year == min_year and month < min_month:
        gtrend_ransomware_dict.pop(key)
    elif year == max_year and month > max_month:
        gtrend_ransomware_dict.pop(key)

In [20]:
# find top 5 keys with highest values
top_10_bitcoin_gtrend = sorted(gtrend_bitcoin_dict, key=gtrend_bitcoin_dict.get, reverse=True)[:10]
top_10_crypto_gtrend = sorted(gtrend_crypto_dict, key=gtrend_crypto_dict.get, reverse=True)[:10]
top_10_ransomware_gtrend = sorted(gtrend_ransomware_dict, key=gtrend_ransomware_dict.get, reverse=True)[:10]

# find top 5 values
top_10_bitcoin_gtrend_values = sorted(gtrend_bitcoin_dict.values(), reverse=True)[:10]
top_10_crypto_gtrend_values = sorted(gtrend_crypto_dict.values(), reverse=True)[:10]
top_10_ransomware_gtrend_values = sorted(gtrend_ransomware_dict.values(), reverse=True)[:10]

## Bitcoin

In [21]:
# find entries in df_transaction_counts which correspond to the top 5 bitcoin gtrend values
df_transaction_counts_sample_top_10_bitcoin = df_transaction_counts_sample[df_transaction_counts_sample['year_month'].isin(top_10_bitcoin_gtrend)]

# perform Z-test on the top 5 bitcoin gtrend values
ls_proportion_ransomware_sample_top_5_bitcoin = df_transaction_counts_sample_top_10_bitcoin['proportion_ransomware'].tolist()
mean_proportion_ransomware_sample_top_5_bitcoin = np.mean(ls_proportion_ransomware_sample_top_5_bitcoin)
std_dev_proportion_ransomware_sample_top_5_bitcoin = np.std(ls_proportion_ransomware_sample_top_5_bitcoin)

sample_size = len(ls_proportion_ransomware_sample_top_5_bitcoin)
sample_mean = mean_proportion_ransomware_sample_top_5_bitcoin
population_mean = mean_proportion_ransomware_sample
population_std_dev = std_dev_proportion_ransomware_sample
alpha = 0.05

z_score = (sample_mean - population_mean) / (population_std_dev / np.sqrt(sample_size))
z_critical = stats.norm.ppf(1 - alpha)

print('z_score:', z_score)
print('z_critical:', z_critical)

if z_score > z_critical:
    print('Reject null hypothesis')
else:
    print('Fail to reject null hypothesis')

z_score: -1.7966304206458878
z_critical: 1.6448536269514722
Fail to reject null hypothesis


In [22]:
# find entries in df_transaction_counts which correspond to the top 5 bitcoin gtrend values
df_transaction_counts_top_10_bitcoin = df_transaction_counts[df_transaction_counts['year_month'].isin(top_10_bitcoin_gtrend)]

# perform Z-test on the top 5 bitcoin gtrend values
ls_proportion_ransomware_top_5_bitcoin = df_transaction_counts_top_10_bitcoin['proportion_ransomware'].tolist()
mean_proportion_ransomware_top_5_bitcoin = np.mean(ls_proportion_ransomware_top_5_bitcoin)
std_dev_proportion_ransomware_top_5_bitcoin = np.std(ls_proportion_ransomware_top_5_bitcoin)

sample_size = len(ls_proportion_ransomware_top_5_bitcoin)
sample_mean = mean_proportion_ransomware_top_5_bitcoin
population_mean = mean_proportion_ransomware
population_std_dev = std_dev_proportion_ransomware
alpha = 0.05

z_score = (sample_mean - population_mean) / (population_std_dev / np.sqrt(sample_size))
z_critical = stats.norm.ppf(1 - alpha)

print('z_score:', z_score)
print('z_critical:', z_critical)

if z_score > z_critical:
    print('Reject null hypothesis')
else:
    print('Fail to reject null hypothesis')

z_score: -0.7097783318440752
z_critical: 1.6448536269514722
Fail to reject null hypothesis


## Crypto

In [23]:
# find entries in df_transaction_counts which correspond to the top 5 crypto gtrend values
df_transaction_counts_sample_top_10_crypto = df_transaction_counts_sample[df_transaction_counts_sample['year_month'].isin(top_10_crypto_gtrend)]

# perform Z-test on the top 5 crypto gtrend values
ls_proportion_ransomware_sample_top_10_crypto = df_transaction_counts_sample_top_10_crypto['proportion_ransomware'].tolist()
mean_proportion_ransomware_sample_top_10_crypto = np.mean(ls_proportion_ransomware_sample_top_10_crypto)
std_dev_proportion_ransomware_sample_top_10_crypto = np.std(ls_proportion_ransomware_sample_top_10_crypto)

sample_size = len(ls_proportion_ransomware_sample_top_10_crypto)
sample_mean = mean_proportion_ransomware_sample_top_10_crypto
population_mean = mean_proportion_ransomware
population_std_dev = std_dev_proportion_ransomware
alpha = 0.05

z_score = (sample_mean - population_mean) / (population_std_dev / np.sqrt(sample_size))
z_critical = stats.norm.ppf(1 - alpha)

print('z_score:', z_score)
print('z_critical:', z_critical)

if z_score > z_critical:
    print('Reject null hypothesis')
else:
    print('Fail to reject null hypothesis')

z_score: -0.7470449531207568
z_critical: 1.6448536269514722
Fail to reject null hypothesis


In [24]:
# find entries in df_transaction_counts which correspond to the top 5 crypto gtrend values
df_transaction_counts_top_10_crypto = df_transaction_counts[df_transaction_counts['year_month'].isin(top_10_crypto_gtrend)]

# perform Z-test on the top 5 crypto gtrend values
ls_proportion_ransomware_top_10_crypto = df_transaction_counts_top_10_crypto['proportion_ransomware'].tolist()
mean_proportion_ransomware_top_10_crypto = np.mean(ls_proportion_ransomware_top_10_crypto)
std_dev_proportion_ransomware_top_10_crypto = np.std(ls_proportion_ransomware_top_10_crypto)

sample_size = len(ls_proportion_ransomware_top_10_crypto)
sample_mean = mean_proportion_ransomware_top_10_crypto
population_mean = mean_proportion_ransomware
population_std_dev = std_dev_proportion_ransomware
alpha = 0.05

z_score = (sample_mean - population_mean) / (population_std_dev / np.sqrt(sample_size))
z_critical = stats.norm.ppf(1 - alpha)

print('z_score:', z_score)
print('z_critical:', z_critical)

if z_score > z_critical:
    print('Reject null hypothesis')
else:
    print('Fail to reject null hypothesis')

z_score: -0.7367439586117552
z_critical: 1.6448536269514722
Fail to reject null hypothesis


## Ransomware

In [25]:
# find entries in df_transaction_counts which correspond to the top 5 ransomware gtrend values
df_transaction_counts_top_10_ransomware_sample = df_transaction_counts_sample[df_transaction_counts_sample['year_month'].isin(top_10_ransomware_gtrend)]

# perform Z-test on the top 10 ransomware gtrend values
ls_proportion_ransomware_sample_top_10_ransomware = df_transaction_counts_top_10_ransomware_sample['proportion_ransomware'].tolist()
mean_proportion_ransomware_sample_top_10_ransomware = np.mean(ls_proportion_ransomware_sample_top_10_ransomware)
std_dev_proportion_ransomware_sample_top_10_ransomware = np.std(ls_proportion_ransomware_sample_top_10_ransomware)

sample_size = len(ls_proportion_ransomware_sample_top_10_ransomware)
sample_mean = mean_proportion_ransomware_sample_top_10_ransomware
population_mean = mean_proportion_ransomware
population_std_dev = std_dev_proportion_ransomware
alpha = 0.05

z_score = (sample_mean - population_mean) / (population_std_dev / np.sqrt(sample_size))
z_critical = stats.norm.ppf(1 - alpha)

print('z_score:', z_score)
print('z_critical:', z_critical)

if z_score > z_critical:
    print('Reject null hypothesis')
else:
    print('Fail to reject null hypothesis')

z_score: -0.11454219308627388
z_critical: 1.6448536269514722
Fail to reject null hypothesis


In [26]:
# find entries in df_transaction_counts which correspond to the top 5 ransomware gtrend values
df_transaction_counts_top_10_ransomware = df_transaction_counts[df_transaction_counts['year_month'].isin(top_10_ransomware_gtrend)]

# perform Z-test on the top 10 ransomware gtrend values
ls_proportion_ransomware_top_10_ransomware = df_transaction_counts_top_10_ransomware['proportion_ransomware'].tolist()
mean_proportion_ransomware_top_10_ransomware = np.mean(ls_proportion_ransomware_top_10_ransomware)
std_dev_proportion_ransomware_top_10_ransomware = np.std(ls_proportion_ransomware_top_10_ransomware)

sample_size = len(ls_proportion_ransomware_top_10_ransomware)
sample_mean = mean_proportion_ransomware_top_10_ransomware
population_mean = mean_proportion_ransomware
population_std_dev = std_dev_proportion_ransomware
alpha = 0.05

z_score = (sample_mean - population_mean) / (population_std_dev / np.sqrt(sample_size))
z_critical = stats.norm.ppf(1 - alpha)

print('z_score:', z_score)
print('z_critical:', z_critical)

if z_score > z_critical:
    print('Reject null hypothesis')
else:
    print('Fail to reject null hypothesis')

z_score: -0.11060596476950808
z_critical: 1.6448536269514722
Fail to reject null hypothesis
